# Straight-ray tomography - Community Experiment

Monday morning, 7:03:51 am. You come running into your office. Breathless. Your boss is already there, sitting relaxed on your desk, looking at you with this slightly sadistic smile. For being 231 seconds too late, he tasks you with the most boring of all jobs: Picking arrival times in the 231 seismic recordings that your company just acquired somewhere in the middle of nowhere. Subsequently, you are ordered to perform a traveltime tomography in order to see if there is anything interesting to discover underground.

## 0. Import some Python packages
 
Being a Python aficionado, you know how to tackle this, and you quickly start by importing a couple of Python packages for matrix-vector operations, for plotting, etc.

In [ ]:
# Some Python packages.

import numpy as np
import scipy
import time

import sys
sys.path.insert(0,'./utils')  # This contains functions to compute G.
from grid import *
from straight_ray_tracer import *

# Set some parameters to make plots nicer.

plt.rcParams["font.family"] = "serif"
plt.rcParams.update({'font.size': 20})

## 1. Measuring traveltimes

So you get to work, picking traveltimes of P waves in somewhat noisy recordings, displayed on your hopelessly outdated computer screen. Click, click, click, ..., click. Done!

In [ ]:
# Load all traveltime observations.
d_obs_1=np.load('d_obs_1.npy')

# Plot traveltime observations.
plt.plot(d_obs_1, 'kx')
plt.ylabel('travel time [s]')
plt.xlabel('ray path index')
plt.title('traveltime')
plt.grid()
plt.show()

## 2. Measuring more traveltimes

Despite being a little monotonous, you somehow like this traveltime picking job. It reminds you of these wonderfully simple computer games that you used to play many years ago; and after all, it is light work. So, you decide to repeat this procedure a couple of times. Click, click, click, ..., until your boss peeks into your office: 

"What the hell ...?!?!"

So, you stop at this point where you have 10 repeated traveltime picks of the whole dataset, and you display what you got:

In [ ]:
# Load all traveltime observations.
d_obs_1=np.load('d_obs_1.npy')
d_obs_2=np.load('d_obs_2.npy')
d_obs_3=np.load('d_obs_3.npy')
d_obs_4=np.load('d_obs_4.npy')
d_obs_5=np.load('d_obs_5.npy')
d_obs_6=np.load('d_obs_6.npy')
d_obs_7=np.load('d_obs_7.npy')
d_obs_8=np.load('d_obs_8.npy')
d_obs_9=np.load('d_obs_9.npy')
d_obs_10=np.load('d_obs_10.npy')

# Compute mean traveltimes.
d_obs_mean=(d_obs_1+d_obs_2+d_obs_3+d_obs_4+d_obs_5+d_obs_6+d_obs_7+d_obs_8+d_obs_9+d_obs_10)/10.0

print(3.0*60.0)

# Plot mean traveltimes.
plt.plot(d_obs_mean,'kx')
plt.ylabel('travel time [s]')
plt.xlabel('ray path index')
plt.title('mean traveltimes')
plt.show()

# Plot traveltime variations relative to the mean.
plt.plot(d_obs_1-d_obs_mean, 'kx')
plt.plot(d_obs_2-d_obs_mean, 'kx')
plt.plot(d_obs_3-d_obs_mean, 'kx')
plt.plot(d_obs_4-d_obs_mean, 'kx')
plt.plot(d_obs_5-d_obs_mean, 'kx')
plt.plot(d_obs_6-d_obs_mean, 'kx')
plt.plot(d_obs_7-d_obs_mean, 'kx')
plt.plot(d_obs_8-d_obs_mean, 'kx')
plt.plot(d_obs_9-d_obs_mean, 'kx')
plt.plot(d_obs_10-d_obs_mean, 'kx')
plt.ylabel('travel time [s]')
plt.xlabel('ray path index')
plt.title('traveltime variations w.r.t mean')
plt.grid()
plt.show()

The mean traveltimes appear reasonable to you. They mostly reflect the length of the ray paths from source to receiver. The traveltime variations with respect to the mean contain some interesting features. Some traveltimes seem to deviate a lot from the mean. They might be outliers, but who knows? And what is an outlier anyway? For measurement indices between around 120 and 170, you observe that one family of traveltime picks is consistently above the mean, while another family of picks is below the mean. Maybe this is because the P wave could not be identified unambiguously. Therefore, in some of the cases, you may have accidentally picked another arrival.

**Exercise 1**: Based on the repeated traveltime measurements, build *one* dataset that you will later use for the inversion. This dataset could be the mean of all traveltimes, one specific dataset among the 10 different ones, or something else that you find meaningful. (Of course, you could, in priciple, also invert different datasets in order to produce different tomographic models. However, your boss is simple-minded, and showing him more than 1 model may stress his mental abilities too much.)

In [ ]:
d_obs = ???

## 3. Numerical setup of the inversion

Having decided what you consider to be *the* dataset, you define the most basic geometric input for the inversion, including the dimensions of the model domain, as well as the positions of sources and receivers. This setup simulates a cross-hole tomography where sources are on one side of the domain, and receivers are on the other one.

In [ ]:
# Define the numerical grid. ---------------------------------------------
dimension=2 # Here we only consider 2D problems anyway.
x_min=0.0 # Minimum x-coordinate  
y_min=0.0 # Minimum y-coordinate
dx=2.5 # Grid spacing in x-direction
dy=2.5 # Grid spacing in y-direction
Nx=20.0 # Number of grid points in x-direction
Ny=20.0 # Number of grid points in y-direction
g = grid(dimension, [x_min,y_min], [dx,dy], np.array([Nx,Ny]))

# Sources and receivers. -------------------------------------------------
src_locations = np.array([ 0.0 * np.ones((11,)), np.linspace(0,50,11)])
rec_locations = np.array([ 50.0 * np.ones((21,)), np.linspace(0,50,21)])

sources, receivers = get_all_to_all_locations(src_locations, rec_locations)
plot_rays(sources, receivers, g)

## 4. Compute forward matrix G

Knowing source and receiver positions, and the setup of the domain, you compute the forward modelling matrix **G** that connects a slowness model **m** to a synthetic data vector **d** via **d**=**Gm**. In addition to computing **G**, you also visualise the ray density and the entries of **G**. Obviously, the ray density is rather uneven, and **G** is pretty sparse.

In [ ]:
# Compute G and measure how long that takes.
tic = time.clock()
G = create_forward_operator(sources, receivers, g)
toc = time.clock()

# Print some statistics of G.
print('Time elapsed:      {:10.4f} s'.format(toc-tic))
print('Matrix shape:            ', G.shape)
print('Data points:             ', G.shape[0])
print('Unknowns in model space: ', G.shape[1])
print('Non-zero entries:        ', G.count_nonzero())
print('Ratio of non-zeros: {:10.4f} %'.format(100 * G.count_nonzero() / (G.shape[0] * G.shape[1])))

# Plot ray density and entries of G.
plot_ray_density(G,g)

print('Sparsity pattern of the forward matrix:')
plt.spy(G, markersize=2)
plt.gca().xaxis.tick_bottom()
plt.xlabel('model space index')
plt.ylabel('data space index')
plt.title(r'non-zero entries of $\mathbf{G}$')
plt.show()

## 5. Design prior covariance matrices

You quickly realise that $\mathbf{G}^T\mathbf{G}$ is not invertible, which forces you to regularise the inverse problem. For this, you have three tuning parameters at your disposition: the prior standard deviation of the traveltime measurements, $\sigma_D$, the prior standard deviation of the model parameters, $\sigma_M$, and the correlation (or smoothing) length $\lambda$. 

**Exercise 2**: Find suitable values for the regularisation parameters. (This is obviously a subjective matter. You may need to run the inversion for different choices of the parameters in order to somehow make a reasonable decision.)

**Note**: The prior mean model is here assumed to be known. In reality, this is of course not the case.

**Note**: The correlation length cannot be chosen arbitrarily. For too large values, the prior model covariance $\mathbf{C}_M$ is not invertible or at least poorly conditioned.

In [ ]:
# Prior covariance parameters. -------------------------------------------
sigma_d = ???         # Data standard deviation.
sigma_m = ???         # Prior model variance (damping).
correlation_length = ??? # Smoothing.


# Prior data covariance matrix. ------------------------------------------
Cd = sigma_d**2 * scipy.sparse.eye(len(d_obs))
Cd_inv = 1 / sigma_d**2 * scipy.sparse.eye(len(d_obs))

# Prior model covariance matrix. -----------------------------------------
Cm = g.get_gaussian_prior(correlation_length)
Cm *= sigma_m
Cm_inv = scipy.sparse.linalg.inv(Cm)


# Prior model. -----------------------------------------------------------
vp = 3000.0 * np.ones(g.npoints) 
m_prior = (1/vp).ravel()
plot_model(m_prior, g, 'prior model')

## 6. Solve inverse problem

You are now equipped with all ingredients needed to solve the inverse problem. For this, you compute the inverse of the Hessian of the least-squares misfit functional, $\mathbf{C}_M^{-1}+\mathbf{G}^T \mathbf{C}_D^{-1} \mathbf{G}$, which is equal to the posterior covariance, $\mathbf{\tilde{C}}_M$.

**Exercise 3**: Run the inversion and discuss the results in the context of the subjective choices you made before. Think about the effect of your chosen dataset on your solution. Furthermore, consider your choices of the regularisation parameters and how these influence the estimated model. How could you better explore the range of regularisation parameters? Are there any physical intuitions that could guide your otherwise ad hoc choices of their values?

In [ ]:
# Hessian ----------------------------------------------------------------
H = G.T * Cd_inv * G + Cm_inv

# Posterior covariance ---------------------------------------------------
Cm_post = scipy.sparse.linalg.inv(H);

# Posterior mean. --------------------------------------------------------
m_est = Cm_post * (G.T * Cd_inv * d_obs + Cm_inv * m_prior)
d_est = G * m_est
d_prior = G * m_prior

# Resolution matrix. -----------------------------------------------------
R = np.identity(g.npoints[0]*g.npoints[1]) - Cm_inv*Cm_post
plt.matshow(R, cmap=plt.cm.get_cmap('seismic'))
plt.clim(-0.4,0.4)
plt.colorbar()
plt.title('resolution matrix')
plt.gca().xaxis.tick_bottom()
plt.show()

print('number of effectively resolved model parameters: %f' % np.trace(R))

# Plot. ------------------------------------------------------------------
plot_model(m_est, g, 'reconstructed slowness')